In [1]:
import warnings
warnings.filterwarnings("ignore")
import numpy as np
import pandas as pd
from pandas import datetime
import matplotlib.pyplot as plt
import seaborn as sns # advanced vizs
%matplotlib inline
from statsmodels.distributions.empirical_distribution import ECDF
from statsmodels.tsa.seasonal import seasonal_decompose
from statsmodels.graphics.tsaplots import plot_acf, plot_pacf
from prophet import Prophet
import logging

sys.path.insert(0,'../scripts/')
from log_helper import Logger_Class
import io
import dvc.api





In [2]:
# repo = 'https://github.com/nahomHmichael/Rossman-Pharmaceutical-Sales-Pridiction'
# rev = 'bc0981'
# path='data/train_store_data.csv'
# data_train =  dvc.api.get_url(path=path,repo=repo,rev=rev)
# train = pd.read_csv(io.StringIO(data_train),low_memory=False)

In [4]:
train = pd.read_csv('../data/train.csv',parse_dates=True, low_memory=False,index_col='Date')
test = pd.read_csv('../data/test.csv',parse_dates=True, low_memory=False, index_col='Date')
store = pd.read_csv('../data/store.csv',low_memory=False)

In [7]:
store.shape

(1115, 10)

In [3]:
# # importing train data to learn
# train = pd.read_csv("../data/train.csv", 
#                     parse_dates = True, low_memory = False, index_col = 'Date')
# store = pd.read_csv("../data/store.csv", 
#                     low_memory = False)
# test = pd.read_csv('../data/test.csv',parse_dates=True, low_memort = False )

DatetimeIndex(['2015-07-31', '2015-07-31', '2015-07-31', '2015-07-31',
               '2015-07-31', '2015-07-31', '2015-07-31', '2015-07-31',
               '2015-07-31', '2015-07-31',
               ...
               '2013-01-01', '2013-01-01', '2013-01-01', '2013-01-01',
               '2013-01-01', '2013-01-01', '2013-01-01', '2013-01-01',
               '2013-01-01', '2013-01-01'],
              dtype='datetime64[ns]', name='Date', length=1017209, freq=None)

In [4]:
# print("In total: ", train.shape)
# train.head(5)

In total:  (1017209, 8)


,Store,DayOfWeek,Sales,Customers,Open,Promo,StateHoliday,SchoolHoliday
Date,,,,,,,,
2015-07-31,1,5,5263,555,1,1,0,1
2015-07-31,2,5,6064,625,1,1,0,1
2015-07-31,3,5,8314,821,1,1,0,1
2015-07-31,4,5,13995,1498,1,1,0,1
2015-07-31,5,5,4822,559,1,1,0,1


In [5]:
# # data extraction
# train['Year'] = train.index.year
# train['Month'] = train.index.month
# train['Day'] = train.index.day
# train['WeekOfYear'] = train.index.weekofyear

# # adding new variable
# train['SalePerCustomer'] = train['Sales']/train['Customers']
# train['SalePerCustomer'].describe()

count    844340.000000
mean          9.493619
std           2.197494
min           0.000000
25%           7.895563
50%           9.250000
75%          10.899729
max          64.957854
Name: SalePerCustomer, dtype: float64

In [ ]:
# sns.set(style = "ticks")# to format into seaborn 
# c = '#386B7F' # basic color for plots
# plt.figure(figsize = (12, 6))

# plt.subplot(311)
# cdf = ECDF(train['Sales'])
# plt.plot(cdf.x, cdf.y, label = "statmodels", color = c);
# plt.xlabel('Sales'); plt.ylabel('ECDF');

# # plot second ECDF  
# plt.subplot(312)
# cdf = ECDF(train['Customers'])
# plt.plot(cdf.x, cdf.y, label = "statmodels", color = c);
# plt.xlabel('Customers');

# # plot second ECDF  
# plt.subplot(313)
# cdf = ECDF(train['SalePerCustomer'])
# plt.plot(cdf.x, cdf.y, label = "statmodels", color = c);
# plt.xlabel('Sale per Customer');